In [1]:
from boknis_eck import BEDatabaseWizard
from spectrum import SfgAverager, DummyPlotter 

from datetime import datetime, date, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.dates import MonthLocator, DateFormatter
from matplotlib.lines import Line2D

%matplotlib qt
plt.style.use("qt.mpltstyle")

# plot functions
def plot_by_time(dataframes, param, leg, scale=1):
    fig, ax = plt.subplots()
    ax2 = ax.twinx()

    months = MonthLocator(range(1, 13), bymonthday=1, interval=6)
    monthsFmt = DateFormatter("%b '%y")
   
    for df in dataframes:
        
        ax.scatter(df["sampling_date"], df["sml_"+param]*scale, color="red")
        ax.scatter(df["sampling_date"], df["bulk_"+param]*scale, color="blue", marker="^")
        ax2.scatter(df["sampling_date"], df["chlorophyll"], color="green", marker="+")

        #ax.plot(df["sampling_date"], df["sml_"+param]*scale, color="red")
        #ax.plot(df["sampling_date"], df["bulk_"+param]*scale, color="blue")
        #ax2.plot(df["sampling_date"], df["chlorophyll"], color="green")


    #for i in range(8, 20, 1):
            #lower = str(date(2000+i, 3, 1))
            #upper = str(date(2000 + i, 9, 1))
            #ax.axvspan(lower, upper, color="gray", alpha=0.4)

    legend_elements = [Line2D([0], [0], marker='^', label='Bulk water',
                              markerfacecolor='blue', mew=0.3,  mec="blue", aa=True, linestyle=''),
                               Line2D([0], [0], marker='o', label='Surface microlayer',
                                       markerfacecolor='red', mew=0.3, mec="red", aa=True, linestyle=''),
                               Line2D([0], [0], marker='+', label='Chlorophyll a',
                                       markerfacecolor='green', mew=2, mec="green", aa=True, linestyle='', markersize=10)
                                ]

    ax.legend(handles=legend_elements)
    ax.xaxis.set_major_locator(months)
    ax.xaxis.set_major_formatter(monthsFmt)
    ax.autoscale_view()
    fig.autofmt_xdate()

    ax.set_xlabel("time ")
    ax.set_ylabel(leg)
    ax2.set_ylabel("Chlorophyll a concentration/\n µg/L")
    rcParams['xtick.labelsize'] = 'small'

def plot_ratio_by_time(dataframes, params):
    fig, ax = plt.subplots()
    ax2 = ax.twinx()

    months = MonthLocator(range(1, 13), bymonthday=1, interval=6)
    monthsFmt = DateFormatter("%b '%y")
   
    for df in dataframes:
        
        if params[1] == "oh":
            sml_total_oh = df["sml_oh1"]+df["sml_oh2"]
            bulk_total_oh = df["bulk_oh1"]+df["bulk_oh2"]     
            sml_ratio = df["sml_"+params[0]]/sml_total_oh
            bulk_ratio = df["bulk_"+params[0]]/bulk_total_oh
            
        else: 
            sml_ratio = df["sml_"+params[0]]/df["sml_"+params[1]]
            bulk_ratio = df["bulk_"+params[0]]/df["bulk_"+params[1]]
        
        ax.scatter(df["sampling_date"], sml_ratio, color="red")
        ax.scatter(df["sampling_date"], bulk_ratio, color="blue", marker="^")
        ax2.scatter(df["sampling_date"], df["chlorophyll"], color="green", marker="+")

        #ax.plot(df["sampling_date"], sml_ratio, color="red")
        #ax.plot(df["sampling_date"], bulk_ratio, color="blue")
        #ax2.plot(df["sampling_date"], df["chlorophyll"], color="green")


    for i in range(8, 20, 1):
            lower = date(2000+i, 3, 1)
            upper = date(2000 + i, 9, 1)
            ax.axvspan(lower, upper, color="gray", alpha=0.4)

    legend_elements = [Line2D([0], [0], marker='^', label='Bulk water',
                              markerfacecolor='blue', mew=0.3,  mec="blue", aa=True, linestyle=''),
                               Line2D([0], [0], marker='o', label='Surface microlayer',
                                       markerfacecolor='red', mew=0.3, mec="red", aa=True, linestyle=''),
                               Line2D([0], [0], marker='+', label='Chlorophyll a',
                                       markerfacecolor='green', mew=2, mec="green", aa=True, linestyle='', markersize=10)
                                ]

    ax.legend(handles=legend_elements)
    ax.xaxis.set_major_locator(months)
    ax.xaxis.set_major_formatter(monthsFmt)
    ax.autoscale_view()
    fig.autofmt_xdate()

    ax.set_xlabel("time ")
    ax.set_ylabel(f'{params[0]}/{params[1]} ratio')
    ax2.set_ylabel("Chlorophyll a concentration/\n µg/L")
    rcParams['xtick.labelsize'] = 'small'

    plt.show()


bz = BEDatabaseWizard()



In [55]:
%matplotlib qt
plt.style.use("qt.mpltstyle")

# todo: filter function returns no type datetime.date 
df = bz.filter_date('2008-01-01', '2015-12-31')
#df["sampling_date"]=df["sampling_date"].apply(lambda x: x.date())
plot_by_time([df], "coverage", "somelegend")


findfont: Font family ['arial'] not found. Falling back to DejaVu Sans.


In [ ]:
# Origin-compatible date
df ['sampling_date'] = df["sampling_date"].dt.strftime('%d.%m.%Y')



In [2]:
def scale_to_polar(array):
    """Normalize the input array to the interva, [0, 2 PI]. Usefull for polar plots."""
    min_ = 1
    max_ = 365
    
    scale = np.pi*2*((array-min_)/(max_-min_))
    return scale




In [17]:
def polar_plot_coverage(records):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='polar')

    # ensure 0° is pointing upwards
    ax.set_theta_zero_location("N")
    # make theta grow clockwise
    ax.set_theta_direction(-1)

    # labels of the theta gridlines set as month abbreviations
    angles = (0, 31, 59.25, 90.25, 120.25, 151.25, 181.25, 212.25, 243.25, 273.25, 304.25, 334.25)
    months = ("Jan", "Feb", "Mar" , "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec" )

    # customize theta gridlines and labels
    lines,label = plt.thetagrids( angles, months)

    for key in records:
        dates = records[key]["sampling_date"].apply(lambda x: x.timetuple().tm_yday)
        dates = scale_to_polar(dates)
        
        c = ax.scatter(dates, records[key]["norm_sml_coverage"], marker="o", color="red")
        c = ax.scatter(dates, records[key]["norm_bulk_coverage"], marker="x", color="blue")
    

In [63]:
def process_years(be_db_wz):
    """Call the normalize_year_records function for each yearly record"""
    out = {}
    years = (i for i in range(2009, 2019) if i != 2016)
    for year in years:
        out[year] = normalize_year_records(be_db_wz.filter_year(year))
    return out

def normalize_year_records(df):
    """Normalize the dataset of a particular year to the corresponding maximum values"""
    
    #todo: implement this for all the other params
    for i in ("sml_coverage", "bulk_coverage", "sml_ch", ""):
        df[f'norm_{i}'] = df[i] / df[i].max()
    return df

In [64]:
%matplotlib qt
q = process_years(bz)
#polar_plot_coverage(q)
q[2014]["sampling_date"].dt.month

for key in q:
    q[key].sort_values("sampling_date", inplace=True)
    plt.scatter(q[key]["sampling_date"].dt.month, q[key]["norm_sml_ch"], marker="o", label=key)
plt.legend()

/home/flo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/flo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
